In [2]:
##### import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random
import torch 

Batch_Size = 50
num_epochs = 200
learning_rate = 1e-5
samples = 20000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


cap = cv2.VideoCapture('video_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<10):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
    
print(np.shape(uframes[0]))
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

for i in range(samples):
    switch = random.randint(0,1)
    if switch==0:
        x = random.randint(0,1)
        if x==0:
            data1.append(np.array(np.transpose(np.asarray(uframes[1]),(2,0,1)),dtype=np.float32))
            data2.append(np.array(np.transpose(np.asarray(uframes[-1]),(2,0,1)),dtype=np.float32))
        elif x==1:
            data1.append(np.array(np.transpose(np.asarray(uframes[2]),(2,0,1)),dtype=np.float32))
            data2.append(np.array(np.transpose(np.asarray(uframes[-1]),(2,0,1)),dtype=np.float32))
        #mg.show()
        
    elif switch==1:
        data1.append(np.array(np.transpose(np.asarray(uframes[0]),(2,0,1)),dtype=np.float32))
        data2.append(np.array(np.transpose(np.asarray(uframes[0]),(2,0,1)),dtype=np.float32))

    



#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data

im1 = Image.open('tree1.png')
im1 = im1.resize((10,10))

im2 = Image.open('tree2.png')
im2 = im2.resize((10,10))

im3 = Image.open('tree3.png')
im3 = im3.resize((10,10))

img = Image.new('RGB', (100,100), color = '#9b7653')
img.paste(im1,(5,75),im1)
img.paste(im1,(10,55),im1)
img.paste(im1,(25,50),im1)
img.paste(im2,(75,5),im2)
img.paste(im2,(60,30),im2)
img.paste(im2,(90,20),im2)
img.paste(im3,(75,60),im3)
img.paste(im3,(60,80),im3)
im_n = np.reshape(img,(100,100,3))
b = np.asarray(im_n)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(20000, 3)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        #out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,100, 100)))
    print("y:", model(y[0].reshape(1,3,100, 100)))
    print("b:", model(b.reshape(1,3,100, 100)))


a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
(100, 100, 3)
(20000, 3, 100, 100)
epoch [1/200], loss:-0.9994
x:  tensor([[

epoch [33/200], loss:-0.9979
x:  tensor([[-1.6385,  4.5365, 18.5236]], grad_fn=<AddmmBackward>)
y: tensor([[-1.6534,  4.6060, 19.0159]], grad_fn=<AddmmBackward>)
b: tensor([[-1.2917,  3.4499, 12.8372]], grad_fn=<AddmmBackward>)
epoch [34/200], loss:-1.0000
x:  tensor([[  1.7061, -10.7692,  12.3391]], grad_fn=<AddmmBackward>)
y: tensor([[  1.7183, -10.9333,  12.7479]], grad_fn=<AddmmBackward>)
b: tensor([[ 0.9581, -6.9885,  8.1933]], grad_fn=<AddmmBackward>)
epoch [35/200], loss:-1.0000
x:  tensor([[ 1.0640, -6.0773, 32.8676]], grad_fn=<AddmmBackward>)
y: tensor([[ 1.0640, -6.0773, 32.8676]], grad_fn=<AddmmBackward>)
b: tensor([[ 0.5572, -4.5002, 23.5738]], grad_fn=<AddmmBackward>)
epoch [36/200], loss:-0.9998
x:  tensor([[ 0.5843, -1.4632,  8.8172]], grad_fn=<AddmmBackward>)
y: tensor([[ 0.5849, -1.4239,  9.2696]], grad_fn=<AddmmBackward>)
b: tensor([[ 0.2115, -0.6794,  5.9367]], grad_fn=<AddmmBackward>)
epoch [37/200], loss:-0.9998
x:  tensor([[ 0.1979, -7.3209,  6.0907]], grad_fn=<Ad

epoch [69/200], loss:-0.9996
x:  tensor([[-0.1226,  1.6670, 15.8775]], grad_fn=<AddmmBackward>)
y: tensor([[-0.1226,  1.6670, 15.8775]], grad_fn=<AddmmBackward>)
b: tensor([[-0.1398,  1.0543, 11.4822]], grad_fn=<AddmmBackward>)
epoch [70/200], loss:-0.9998
x:  tensor([[ 0.2297, -1.8273, 20.2538]], grad_fn=<AddmmBackward>)
y: tensor([[ 0.2297, -1.8273, 20.2538]], grad_fn=<AddmmBackward>)
b: tensor([[ 0.1376, -1.2851, 14.4130]], grad_fn=<AddmmBackward>)
epoch [71/200], loss:-0.9999
x:  tensor([[-1.7011, -1.5636, 19.9424]], grad_fn=<AddmmBackward>)
y: tensor([[-1.7011, -1.5636, 19.9424]], grad_fn=<AddmmBackward>)
b: tensor([[-1.2111, -1.0631, 14.2016]], grad_fn=<AddmmBackward>)
epoch [72/200], loss:-0.9997
x:  tensor([[-11.9870,  -3.9417,  12.3958]], grad_fn=<AddmmBackward>)
y: tensor([[-11.9870,  -3.9417,  12.3958]], grad_fn=<AddmmBackward>)
b: tensor([[-7.7040, -2.6940,  9.1196]], grad_fn=<AddmmBackward>)
epoch [73/200], loss:-1.0000
x:  tensor([[ 0.8892, -2.2938, 14.9043]], grad_fn=<Ad

epoch [105/200], loss:-1.0000
x:  tensor([[-1.9872, -3.2035, 17.5835]], grad_fn=<AddmmBackward>)
y: tensor([[-2.0542, -3.2333, 17.9421]], grad_fn=<AddmmBackward>)
b: tensor([[-1.4931, -2.2195, 11.0705]], grad_fn=<AddmmBackward>)
epoch [106/200], loss:-1.0000
x:  tensor([[12.1658, -0.9720, 24.2005]], grad_fn=<AddmmBackward>)
y: tensor([[12.1445, -0.9755, 24.5509]], grad_fn=<AddmmBackward>)
b: tensor([[ 8.0434, -0.7534, 15.4605]], grad_fn=<AddmmBackward>)
epoch [107/200], loss:-0.9993
x:  tensor([[ 1.4950, -1.2055, 23.4986]], grad_fn=<AddmmBackward>)
y: tensor([[ 1.4888, -1.2484, 23.8653]], grad_fn=<AddmmBackward>)
b: tensor([[ 0.9724, -0.9937, 14.8479]], grad_fn=<AddmmBackward>)
epoch [108/200], loss:-0.9998
x:  tensor([[ 1.0447, -3.2385, 19.8962]], grad_fn=<AddmmBackward>)
y: tensor([[ 1.0302, -3.2254, 20.2640]], grad_fn=<AddmmBackward>)
b: tensor([[ 0.6849, -2.2457, 12.5941]], grad_fn=<AddmmBackward>)
epoch [109/200], loss:-0.9980
x:  tensor([[ 0.9064, -1.2503, 25.9948]], grad_fn=<Add

epoch [141/200], loss:-0.9999
x:  tensor([[ 2.5941, -1.2152, 23.1501]], grad_fn=<AddmmBackward>)
y: tensor([[ 3.0830, -1.2214, 23.5022]], grad_fn=<AddmmBackward>)
b: tensor([[ 1.8487, -0.8742, 14.3469]], grad_fn=<AddmmBackward>)
epoch [142/200], loss:-0.9975
x:  tensor([[ 1.9165, -1.3265, 12.5424]], grad_fn=<AddmmBackward>)
y: tensor([[ 1.9194, -1.3210, 12.8819]], grad_fn=<AddmmBackward>)
b: tensor([[ 1.2318, -0.9337,  7.3289]], grad_fn=<AddmmBackward>)
epoch [143/200], loss:-1.0000
x:  tensor([[ 1.2660, -1.0710, 20.2504]], grad_fn=<AddmmBackward>)
y: tensor([[ 1.2670, -1.0670, 20.5942]], grad_fn=<AddmmBackward>)
b: tensor([[ 0.7515, -0.7641, 12.3325]], grad_fn=<AddmmBackward>)
epoch [144/200], loss:-0.9998
x:  tensor([[ 1.1932, -0.8685, 19.1680]], grad_fn=<AddmmBackward>)
y: tensor([[ 1.1920, -0.8693, 19.5091]], grad_fn=<AddmmBackward>)
b: tensor([[ 0.6998, -0.6295, 11.7543]], grad_fn=<AddmmBackward>)
epoch [145/200], loss:-0.9995
x:  tensor([[ 2.7562, -6.4178, 11.5983]], grad_fn=<Add

epoch [177/200], loss:-0.9999
x:  tensor([[ 4.3387, -1.2770, 15.2735]], grad_fn=<AddmmBackward>)
y: tensor([[ 4.3387, -1.2770, 15.2735]], grad_fn=<AddmmBackward>)
b: tensor([[ 2.7836, -0.9309, 10.0970]], grad_fn=<AddmmBackward>)
epoch [178/200], loss:-0.9997
x:  tensor([[ 2.0092, -0.2511, 18.2720]], grad_fn=<AddmmBackward>)
y: tensor([[ 2.0092, -0.2511, 18.2720]], grad_fn=<AddmmBackward>)
b: tensor([[ 1.2766, -0.2186, 12.0666]], grad_fn=<AddmmBackward>)
epoch [179/200], loss:-0.9995
x:  tensor([[ 1.5183, -1.4277, 16.5022]], grad_fn=<AddmmBackward>)
y: tensor([[ 1.4378, -1.5440, 16.8142]], grad_fn=<AddmmBackward>)
b: tensor([[ 0.9305, -1.0885,  9.5222]], grad_fn=<AddmmBackward>)
epoch [180/200], loss:-0.9997
x:  tensor([[ 1.0046, -0.5391, 12.5979]], grad_fn=<AddmmBackward>)
y: tensor([[ 1.0038, -0.5407, 12.9228]], grad_fn=<AddmmBackward>)
b: tensor([[ 0.6024, -0.4476,  7.0295]], grad_fn=<AddmmBackward>)
epoch [181/200], loss:-0.9987
x:  tensor([[ 0.2405, -2.9415, 11.5647]], grad_fn=<Add

In [26]:
data3=[]

class_data = []

for i in range(75):
    img = uframes[0]
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),0))



In [27]:
data3=[]

for i in range(75):
    img = uframes[1]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),2))
    
print(len(class_data))

150


In [28]:
data3=[]

for i in range(75):
    img = uframes[2]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),3))
    
print(len(class_data))

225


In [25]:
print(np.shape(uframes))

(4, 100, 100, 3)


In [29]:
data3=[]

for i in range(75):
    img = uframes[3]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),1))
    
print(len(class_data))

300


In [31]:
import random
p=0
class_data = []
while(p<len(uframes)):
    data3=[]
    print(p)
    for i in range(50):
        img = uframes[p]
        data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
        y3 = np.array(data3)
        x_1 = torch.from_numpy(y3)
        temp = model(x_1[i].reshape(1,3,100,100))
        class_data.append((list(temp.detach().numpy()[0]),p))
    p+=1
    print(len(class_data))

random.shuffle(class_data)
random.shuffle(class_data)

0
50
1
100
2
150
3
200


In [32]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets


def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out


# import some data to play with

# Take the first two features. We could avoid this by using a two-dim dataset
X = [(i[0]) for i in class_data]
y = [i[1] for i in class_data]
#print(X,np.shape(X))
#print(y,np.shape(y))
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
clf = svm.SVC(kernel='linear', C=C)
clf = clf.fit(X, y)
data3=[]
for i in range(4):
    img = uframes[i]
    ms = Image.fromarray(img)
    ms.show()
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))
    temp = temp.detach().numpy()
    print(temp)
    z = clf.predict(temp)
    print(z)

[[ 0.9245042  -0.93822163 12.585339  ]]
[0]
[[ 1.0329318  -0.95168304 14.47444   ]]
[1]
[[ 1.0462801 -0.9146041 14.478038 ]]
[2]
[[ 1.0161911 -0.961267  14.779221 ]]
[3]


In [ ]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

In [ ]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(5,5),im_drone)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

In [ ]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(72,65),im_drone)
    img.paste(imb,(80,80),imb)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)